In [1]:
%pwd

'/home/anonymous/PycharmProjects/MLOps_cv/research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/anonymous/PycharmProjects/MLOps_cv'

In [5]:
from dataclasses import dataclass, field
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    """Configuration for data ingestion."""
    root_dir: Path
    """Path to the directory containing the data."""
    source_URL: str
    """URL of the data source."""
    local_data_file: Path
    """Path to the local file containing the data."""
    unzip_dir: Path
    """Path to the directory containing the unzipped data."""

In [6]:
!ls

app.py	   Dockerfile  main.py	    requirements.txt  src	   templetes
artifacts  dvc.yaml    params.yaml  research	      template.py
config	   env	       README.md    setup.py	      templates


In [7]:
from mlopsCNN.constants import *

In [8]:
from mlopsCNN.constants import *
from mlopsCNN.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
    

In [10]:
import os
import urllib.request as request
import zipfile
from mlopsCNN import logger
from mlopsCNN.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                self.config.source_URL,
                filename=self.config.local_data_file)
            
            logger.info(f"Downloaded {filename} with following info: \n  {headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists")


    def extract_zip_file(self):
        """
        zip_file_path = str
        Extracts the zip file to the local_data_file directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path )

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-28 15:58:22,310: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-28 15:58:22,316: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 15:58:22,317: INFO: common: created directory at: artifacts]
[2023-09-28 15:58:22,318: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-28 15:58:22,319: INFO: 3862079018: File artifacts/data_ingestion/data.zip already exists]


In [36]:
data_ingestion.extract_zip_file

<bound method DataIngestion.extract_zip_file of <__main__.DataIngestion object at 0x7f71605568c0>>